In [33]:
import numpy  as  np 
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [38]:
def fprehook(self, inputs):
    print(f"layer name: {self}")
    print(f"input shape: {inputs[0].shape}")

In [35]:
class SimpleCNN(nn.Module):
    def __init__(self,input_shape = (3,28,28)):
        super(SimpleCNN ,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3 ,out_channels=12,kernel_size=3 , stride=1 ,padding=1)
        handle_1 = self.conv1.register_full_backward_pre_hook(fprehook)
        self.conv2 = nn.Conv2d(in_channels=12 ,out_channels=32,kernel_size=3 , stride=1 ,padding=1)
        handle_2 = self.conv2.register_full_backward_pre_hook(fprehook)
        self.conv3 = nn.Conv2d(in_channels=32 ,out_channels=64,kernel_size=3 , stride=1 ,padding=1)
        handle_3 = self.conv3.register_full_backward_pre_hook(fprehook)
        self.conv4 = nn.Conv2d(in_channels=64 ,out_channels=128,kernel_size=3 , stride=1 ,padding=1)
        handle_4 = self.conv4.register_full_backward_pre_hook(fprehook)
        
        # Flattening the  conv layer 
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(self._get_flatten_layer(input_shape),128)
        self.fc2 = nn.Linear(128,10)


    
    def _get_flatten_layer(self,shape):
        # print("SHAPE" ,shape)
        dummy_input = torch.rand(1 *shape)
        x = self.conv1(dummy_input)
        # print("SHAPE OF CONV1",x.shape)
        x = F.relu(x)
        x = self.conv2(x)
        # print("SHAPE OF CONV2",x.shape)
        x = F.relu(x)
        x = self.conv3(x)
        # print("SHAPE OF CONV3",x.shape)
        x = F.relu(x)
        x = self.conv4(x)
        # print("SHAPE OF CONV4",x.shape)
        x = F.relu(x)
        n_size = self.flatten(x).shape[1]
        print("N_SIZE",n_size)
        return n_size


In [36]:
model = SimpleCNN(input_shape=(3,28,28))
print(model)

SHAPE (3, 28, 28)
SHAPE OF CONV1 torch.Size([12, 28, 28])
SHAPE OF CONV2 torch.Size([32, 28, 28])
SHAPE OF CONV3 torch.Size([64, 28, 28])
SHAPE OF CONV4 torch.Size([128, 28, 28])
N_SIZE 784
SimpleCNN(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
